In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, '..//src/')

import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import icecream as ic

sns.set_palette(sns.color_palette("colorblind", 15))
plt.rcParams['font.family'] = 'Times New Roman'
font_size = 12
params = {'axes.labelsize': font_size+2,
          'axes.titlesize':font_size+4,
          'legend.fontsize': font_size, 
          'xtick.labelsize': font_size,
          'ytick.labelsize': font_size,
          'font.size': font_size}
plt.rcParams.update(params)
sns.set_style("whitegrid")
export_fig_format = "pdf"
fig_format = "png"
dpi = 300
figsize = (12, 3.5)
figsize_map = (16, 8)

markers = ['P', 'o', '^', 's', '*', 'v']
linewidth = 0.1
edgecolor = 'black'

import main
from models import thermal_model
from common import functions, enums, schema
from data import source


In [3]:
PATH_GB_DATA = Path(r'C:\Users\sceac10\OneDrive - Cardiff University\04 - Projects\00 - Final data\Annual_demand')
PATH_CIBSE_DATA = Path(r"C:\Users\sceac10\OneDrive - Cardiff University\General\data\CIBSE weather data\WD16SET\WD16SET\WDD16SET\WDD16SET\WMD16SET\WMD16SET")
PATH_RESULTS = Path(r'C:\Users\sceac10\OneDrive - Cardiff University\General\04 - Analysis')

In [4]:
temp_dataf = functions.get_LSOA_code_to_LADCD_lookup()
temp_dataf = temp_dataf.set_index(schema.geoLookupSchema.lsoa)

In [ ]:
residential_data = main.import_thermal_characteristics_data(main.PATH_GB_DATA / 'Thermal_characteristics_afterEE.csv', 2020, 2050)

In [ ]:
filt = residential_data[schema.DwellingDataSchema.LADCD] == 'S12000013'
residential_data.loc[filt, :]


In [ ]:
residential_data.loc[filt, schema.DwellingDataSchema.THERMAL_LOSSES].hist(bins=10)

In [ ]:
filt = residential_data[schema.DwellingDataSchema.CIBSE_CITY] == 'London_LHR'
residential_data[filt]

In [ ]:
pathlist = Path(PATH_CIBSE_DATA).rglob('*_DSY2_2050High50*.csv')

for path in pathlist:
  print(path)
  print(f"{path.stem.split('_DSY2_')[0]},")

In [ ]:
LA_str = "Cardiff"
column_names = [
  'Year', 'Month', 'Day', 'Hour', 'PWC', 'Cloud', 'DBT', 'WBT', 'RH', 'Press', 'WD', 'WS', 'GSR', 'DSR', 'Alt', 'Dec', 'Cloud1', 'DBT1', 'WBT1', 'Press1', 'WD1', 'WS1'
]
pathlist = Path(PATH_CIBSE_DATA).rglob('Cardiff_DSY2_2050High50*.csv')

list_files = []
for path in pathlist:
  print(path.stem)
  list_files.append(path.stem)
  temp_dataf = pd.read_csv(path, skiprows=32, header=None, delimiter=",", names=column_names)

In [ ]:
external_data = functions.format_weather_data(temp_dataf)

In [ ]:
residential_data.head()

In [ ]:
residential_data.iloc[211857]

In [ ]:
LA_str  = "Cardiff"

filt = (residential_data[schema.DwellingDataSchema.LOCAL_AUTHORITY]==LA_str) & (residential_data[schema.DwellingDataSchema.THERMAL_CAPACITY_LEVEL]=="medium")
LA_residential_data = residential_data.loc[filt, :]
data_source = source.SimulationData(external_data)

sim_dataf = data_source.create_CIBSE_based_simulation_data()

row = LA_residential_data.iloc[0]
results_df = functions.run_simulation(sim_dataf.copy(), row, initial_indoor_air_temperature=21)
print(functions.print_heating_and_cooling_demand(results_df))

In [ ]:
results_df[schema.DataSchema.HEATINGOUTPUT].plot()

In [ ]:
results_df[schema.DataSchema.IAT].plot()

In [ ]:
results_df[[schema.DataSchema.IAT,schema.DataSchema.OAT]].plot()

In [ ]:
column_names = [
  'Year', 'Month', 'Day', 'Hour', 'PWC', 'Cloud', 'DBT', 'WBT', 'RH', 'Press', 'WD', 'WS', 'GSR', 'DSR', 'Alt', 'Dec', 'Cloud1', 'DBT1', 'WBT1', 'Press1', 'WD1', 'WS1'
]
pathlist = Path(PATH_CIBSE_DATA).rglob('Cardiff_DSY2_2050High50*.csv')

list_files = []
for path in pathlist:
  # ic.ic(path.stem)
  cibse_city = path.stem.split('_')[0]
  ic.ic(cibse_city)
  # list_files.append(path.stem)
  temp_dataf = pd.read_csv(path, skiprows=32, header=None, delimiter=",", names=column_names)
  weather_data = functions.format_weather_data(temp_dataf)
  data_source = source.SimulationData(weather_data)
  sim_dataf = data_source.create_CIBSE_based_simulation_data()
  filt_las = (residential_data[schema.DwellingDataSchema.CIBSE_CITY]==cibse_city)
  list_las = list(residential_data.loc[filt_las, schema.DwellingDataSchema.LOCAL_AUTHORITY].unique())
  for LA_str in list_las:
    ic.ic(LA_str)
    filt = ((residential_data[schema.DwellingDataSchema.LOCAL_AUTHORITY]==LA_str) 
            & (residential_data[schema.DwellingDataSchema.THERMAL_CAPACITY_LEVEL]=="medium"))
    LA_residential_data = residential_data.loc[filt, :]

    summary_results, metadata = functions.run_batch_simulation(LA_residential_data.head(), sim_dataf, PATH_RESULTS)
    # save_summary_results_path =  PATH_RESULTS/f'summary_results'
    # save_summary_results_path.mkdir(parents=True, exist_ok=True)
    # summary_results.to_parquet(save_summary_results_path/f'{LA_str}_summary_results.gzip')
    # save_metadata_path =  PATH_RESULTS/f'metadata'
    # save_metadata_path.mkdir(parents=True, exist_ok=True)
    # metadata.to_parquet(save_metadata_path/f'{LA_str}_metadata.gzip')

  

In [ ]:
LA_residential_data

In [ ]:
temp_concat_results = pd.concat([LA_residential_data, summary_results], axis=1)
temp_concat_results.head()

In [ ]:
cooling_per_m2 = schema.ResultSchema.SPECIFICCOOLINGDEMAND.replace("kWh/dwelling", "kWh/m2")
temp_concat_results[cooling_per_m2] = temp_concat_results[schema.ResultSchema.SPECIFICCOOLINGDEMAND]/temp_concat_results["Average floor area m2"]
temp_concat_results[cooling_per_m2]

In [ ]:
nb_dwellings = temp_concat_results["Number of dwellings"].sum()
total_floor_area = temp_concat_results["Average floor area m2"].sum()*nb_dwellings
print(f"In {LA_str}, there are {nb_dwellings:,.0f} dwellings for a total floor area of {total_floor_area/1_000_000:,.0f} km2")

In [ ]:
(-temp_concat_results.groupby("Dwelling forms").agg({cooling_per_m2:"mean"})).plot.bar(rot=0)


In [ ]:
(-temp_concat_results.groupby("Dwelling forms").agg({schema.ResultSchema.COOLINGDEMAND:"sum"})).plot.bar(rot=0)

In [ ]:
(-temp_concat_results.groupby("Dwelling forms").agg({schema.ResultSchema.SPECIFICCOOLINGDEMAND:"mean"})).plot.bar(rot=0)

In [ ]:
temp_LA_results[schema.ResultSchema.COOLINGDEMAND].sum